In [53]:
import os
import json
import csv
import numpy as np

from neurosim.utils.agg import _extract_hpsteps

MODELS = {
    's6_before': {
        'path': 'results/seedrun_m1-2022-01-16/run_seed1394398',
        'eval_idx': 0,
        'name': 'Seed-6 after initialization'
    },
    's6_stdprl': {
        'path': 'results/seedrun_m1-2022-01-16/run_seed1394398/continue_1/continue_1/continue_2/continue_2/continue_2',
        'eval_idx': 1,
        'full': 'results/seedrun_m1-2022-01-16/run_seed1394398/continue_1/continue_1/continue_2/continue_2/continue_2/continue_2/continue_2',
        'name': 'Seed-6 after STDP-RL training'
    },
#     's6_evol_b1': {
#         'path': 'results/20220123-EVOL_b1-goodseed/continue_1/continue_1/continue_1/continue_1',
#         'eval_idx': 0,
#         'iters': 5000,
#         'name': 'Seed-6 after EVOL training'
#     },
    's6_evol_b5': {
        'path': 'results/20220129-EVOL_b5-goodseed/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1',
        'eval_idx': 1,
        'iters': 1600,
        'name': 'Seed-6 after EVOL training'
    },
#     's6_evolstdp': {
#         'path': 'results/evol-stdp-rl_2022-01-21/STDP_ES_best_beta1_pop10.small',
#         'eval_idx': 58,
#         'iters': 1500,
#         'name': 'Seed-6 after EVOL+STDP-RL training'
#     },
#     's6_evolstdp_b5': {
#         'path': 'results/evol-stdp-rl_2022-01-21/STDP_ES_best_beta5_pop10.small',
#         'eval_idx': 59,
#         'iters': 1500
#     },
    's3_before': {
        'path': 'results/seedrun_m1-2022-01-16/run_seed5397326',
        'eval_idx': 0,
        'name': 'Seed-3 after initialization'
    },
    's3_stdprl': {
        'path': 'results/seedrun_m1-2022-01-16/run_seed5397326/continue_1/continue_1/continue_2/continue_2/continue_2',
        'eval_idx': 50,
        'full': 'results/seedrun_m1-2022-01-16/run_seed5397326/continue_1/continue_1/continue_2/continue_2/continue_2/continue_2/continue_2',
        'name': 'Seed-3 after STDP-RL training'
    },
#     's3_evol_b1': {
#         'path': 'results/20220124-EVOL_b1-badseed/continue_1/continue_1/continue_1/continue_1/continue_1',
#         'eval_idx': 6,
#         'iters': 5000,
#         'name': 'Seed-3 after EVOL training'
#     },
    's3_evol_b5': {
        'path': 'results/20220128-EVOL_b5-badseed/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1/continue_1',
        'eval_idx': 2,
        'iters': 1600,
        'name': 'Seed-3 after EVOL training'
    },
#     's3_evolstdp': {
#         'path': 'results/evol-stdp-rl_2022-01-21/STDP_ES_worst_beta1_pop10.small',
#         'eval_idx': 56,
#         'iters': 1500,
#         'name': 'Seed-3 after EVOL+STDP-RL training'
#     },
#     's3_evolstdp_b5': {
#         'path': 'results/evol-stdp-rl_2022-01-21/STDP_ES_worst_beta5_pop10.small',
#         'eval_idx': 52,
#         'iters': 1500
#     }
}



In [57]:
table = [['Model', 'Average on Test Dataset', 'Mean on Test Dataset', 'Trained Episodes', 'Iterations', 'Population']]

def get_eval(model):
    eval_path = os.path.join('../', model['path'], 'evaluation_{}'.format(model['eval_idx']))
    with open(os.path.join(eval_path, 'backupcfg_sim.json')) as f:
        j = json.load(f)
        assert j['env']['seed'] == 42
    acts = []
    with open(os.path.join(eval_path, 'ActionsPerEpisode.txt')) as f:
        for r in csv.reader(f, delimiter='\t'):
            acts.append(int(float(r[1])))
#     assert len(acts) == 100
    return acts

def get_episodes_stdprl(model):
    wdirs, configs = _extract_hpsteps(model['full'], '../')
    total_episodes = 0
    for wdir in wdirs:
        with open(os.path.join('../', wdir, 'ActionsPerEpisode.txt')) as f:
            total_episodes += len(f.read().split('\n'))
    return total_episodes

for model_name, model in MODELS.items():
    acts = get_eval(model)
    
    if 'evol' in model_name:
        population = '10'
        iterations = str(model['iters'])
        mul = 5 if 'b5' in model_name else 1
        episodes = int(population) * int(iterations) * mul
    elif 'stdprl' in model_name:
        population = '-'
        iterations = ''
        episodes = get_episodes_stdprl(model)
    else:
        population = ''
        iterations = ''
        episodes = '0'    
    
    row = [model['name'], np.mean(acts), np.median(acts), episodes, iterations, population]
    
    table.append(row)

In [58]:
table

[['Model',
  'Average on Test Dataset',
  'Mean on Test Dataset',
  'Trained Episodes',
  'Iterations',
  'Population'],
 ['Seed-6 after initialization', 23.38, 19.5, '0', '', ''],
 ['Seed-6 after STDP-RL training', 144.67, 130.5, 4226, '', '-'],
 ['Seed-6 after EVOL training', 499.42, 500.0, 80000, '1600', '10'],
 ['Seed-3 after initialization', 23.04, 19.0, '0', '', ''],
 ['Seed-3 after STDP-RL training', 64.14, 53.0, 8577, '', '-'],
 ['Seed-3 after EVOL training', 499.09, 500.0, 80000, '1600', '10']]

In [56]:
import csv

with open('perf.csv', 'w') as out:
    writer = csv.writer(out)
    for line in table:
        writer.writerow(line)